In [1]:
from sympy import Symbol, Matrix, symbols, sin, lambdify
import time
import  numpy as np
from collections import OrderedDict


In [2]:
id_wind=False
bo=True


In [3]:
for used_logged_v_in_model in [True, False]:
    t0=time.time()

    vlog_i,vlog_j,vlog_k=symbols("vlog_i,vlog_j,vlog_k",real=True)
    v_log=Matrix([[vlog_i],
            [vlog_j],
            [vlog_k]])
    vpred_i,vpred_j,vpred_k=symbols("vpred_i,vpred_j,vpred_k",real=True)  
    v_pred=Matrix([[vpred_i],
            [vpred_j],
            [vpred_k]])
    v_i,v_j,v_k=(vlog_i,vlog_j,vlog_k) if used_logged_v_in_model else (vpred_i,vpred_j,vpred_k)
    v_B=Matrix([[v_i],
            [v_j],
            [v_k]])

    print("\nElapsed : %f s , Prev step time: -1 s \\ Generating first symbols ..."%(time.time()-t0))
    dt=Symbol('dt',positive=True,real=True)
    CL_1_sa = Symbol('C^{sa}_{L,1}',real=True)             # Coeff rechercher
    CD_0_sa = Symbol('C^{sa}_{D,0}',real=True)             # Coeff rechercher
    CD_1_sa = Symbol('C^{sa}_{D,1}',real=True)             # Coeff rechercher
    CD_0_fp = Symbol('C^{fp}_{D,0}',real=True)             # Coeff rechercher
    CD_1_fp = Symbol('C^{fp}_{D,1}',real=True)             # Coeff rechercher                      
    k_0 = Symbol('k_0', real=True)                         # coeff rechercher
    k_1 = Symbol('k_1', real=True)                         # coeff rechercher
    k_2 = Symbol('k_2', real=True)                         # coeff rechercher
    delta_s = Symbol('delta_s', real=True)                # Coeff rechercher : largeur du stall
    alpha_s = Symbol('alpha_s',real=True)                 # Coeff rechercher
    B_B       = Matrix([[1,0,0], [0,1,0], [0,0,1]])                              # Base dans le repère body
    omega1, omega2, omega3 = symbols('\omega_1, \omega_2, \omega_3', real=True)
    Omega     = Matrix([omega1, omega2, omega3])                                 # Vecteur de rotation
    r00, r01, r02, r10, r11, r12, r20, r21, r22 = symbols('r_{00}, r_{01}, r_{02}, r_{10}, r_{11}, r_{12}, r_{20}, r_{21}, r_{22}', real=True)
    R         = Matrix([[r00,r01,r02], [r10,r11, r12], [r20, r21, r22]])          # Matrice de rotation
#    Vb1,Vb2,Vb3=symbols('V_{b1} V_{b2} V_{b3}',real=True)
    #v_B        = Matrix([Vb1, Vb2, Vb3])                                          # Vitesse du corps (repère drone)
    Vw1,Vw2,Vw3=symbols('V_{w1} V_{w2} V_{w3}',real=True)                         # Vitesse du vent dans le repère NED 
    v_W        = Matrix([Vw1, Vw2, Vw3])
    xcp, ycp, zcp = symbols('x_{cp}, y_{cp}, z_{cp}')
    X_cp     = Matrix([xcp, ycp, zcp])                                            # Position du centre de poussé d'un corps dans le repère body
    C_t, C_q, C_h=symbols('C_t,C_q, C_h',real=True)                           # Coefficient de poussée des moteurs, coefficient de couple des moteurs
    motor_axis_in_body_frame = Matrix([1,0,0])                                    # Axe des moteurs, ici placé en mode avion
    omega_rotor = symbols('\omega_{rotor}', real=True)                            # Vitesse de rotation des moteurs
    crossward_B = B_B[:,1]
    c45, s45 = np.cos(45*np.pi/180), np.sin(45*np.pi/180)
    r = np.array(((1,0, 0),(0,c45,-s45),(0,s45, c45)))
    r_neg     = np.array(((1,0, 0), (0,c45, s45),(0,-s45, c45)))
    R_list_sympy = [R, R,  R * r,  R *r_neg, R]      # Liste des matrices de rotation de chaque surface portante du drone, seul les éléments de la queue (element 3 et 4) ne sont pas dans le repère inertiel. 
    cp1x,cp1y, cp1z, cp2x,cp2y,cp2z, cp3x,cp3y,cp3z, cp4x,cp4y,cp4z,cp5x,cp5y,cp5z = symbols('cp1x,cp1y, cp1z, cp2x,cp2y,cp2z, cp3x,cp3y,cp3z, cp4x,cp4y,cp4z,cp5x,cp5y,cp5z', real=True)
    cp_list = [Matrix([cp1x,cp1y, cp1z]), Matrix([cp2x,cp2y,cp2z]), Matrix([cp3x,cp3y,cp3z]), Matrix([cp4x,cp4y,cp4z]), Matrix([cp5x,cp5y,cp5z])]
    A1, A2, A3 = symbols('A_1 A_2 A_3', real=True)
    Aire_list = [A1, A1, A2, A2, A3]  # Liste des 1/2 * rho * S pour chaque surface
    cp_list_rotor = [Matrix([0.713,0.475,0]), Matrix([0.713,-0.475,0])]
    spinning_sense_list = [1,-1]

    ##### Listes des angles (d'attaque et de contrôles) pour faire la somme des forces en une seule équations
    alpha1, alpha2, alpha3, alpha4, alpha5 = symbols('alpha_1, alpha_2, alpha_3, alpha_4, alpha_5', real=True)
    alpha_list =Matrix([alpha1, alpha2, alpha3, alpha4, alpha5])
    alpha0 = Symbol('alpha_0', real=True)
    alpha0_list = Matrix([alpha0 , alpha0 ,0, 0, alpha0])
    delta0_1, delta0_2, delta0_3, delta0_4, delta0_5 = symbols('delta_0_1, delta_0_2, delta_0_3, delta_0_4, delta_0_5', real=True)
    delta0_list = Matrix([delta0_1, delta0_2, delta0_3, delta0_4, delta0_5])
    t1=time.time()
    print("Elapsed : %f s , Prev step time: -1 s \\ Generating dynamics function ..."%(t1-t0))
    def compute_alpha(dragDirection, liftDirection, frontward_Body, VelinLDPlane, R=None):
        frontward_Body if R is not None  else R*frontward_Body 
        calpha= np.vdot(dragDirection, frontward_Body)
        absalpha= -np.arccos(calpha)
        signalpha = np.sign(np.vdot(liftDirection, frontward_Body)) 
        if np.linalg.norm(VelinLDPlane)>1e-7 :
            alpha = signalpha*absalpha 
        else :
            alpha=0
        if abs(alpha)>0.5*np.pi:
            if alpha>0 :alpha=alpha-np.pi 
            else: alpha=alpha+np.pi         
        return alpha

    ##################################################### génération des équations pour Cd et Cl (utiliser pour générer les équations symbolique pour chaque surface portantes) ####################################################
    def compute_cl_cd(a, a_0, a_s, d_0, d_s, cl1sa, cd1fp, k0, k1, k2, cd0fp, cd0sa, cd1sa):
        if bo == True:
            CL_sa = 1/2 * cl1sa * (2*(a + (d_0)))
            CD_sa =   cd1sa * (a + d_0)*(a + d_0)

            C_L = CL_sa + (d_0)
            C_D = CD_sa 
        else:
            CL_sa = 1/2 * cl1sa * sin(2*(a + (k1*d_0) + a_0))
            CD_sa = cd0sa + cd1sa * sin(a + (k0*d_0) + a_0)*sin(a +  (k0*d_0) + a_0)

            CL_fp = 1/2 * cd1fp * sin(2*(a+ (k1*d_0) + a_0))
            CD_fp = cd0fp + cd1fp * sin(a + (k0*d_0) + a_0)*sin(a +  (k0*d_0) + a_0)

            puiss=5
            s = 1.0 - ((a+a_0)**2/a_s**2)**puiss/(((a+a_0)**2/a_s**2)**puiss + 100+200*d_s)

            C_L = CL_fp + s*(CL_sa - CL_fp) + k2 * sin(d_0)
            C_D = CD_fp + s*(CD_sa - CD_fp)
        return C_L, C_D

    def GenDirectForceWing(Omega, cp, vB, vW, R, crossward_Body):
        if bo==True:
        # Cette fonction permet d'obtenir les directions des efforts de portances et de trainé en fonction des vitesses, et de l'orientation dans le repère NED.
            Air_speed_earth = vB -vW
            VelinLDPlane    = Air_speed_earth - Air_speed_earth.dot(R*crossward_Body) * crossward_Body
            dragDirection = -VelinLDPlane / VelinLDPlane.norm()  
            liftDirection = -(R*crossward_Body).cross(dragDirection) 
        else:
            Air_speed_earth = vB - vW 
            Air_speed_body  = (R.T* Air_speed_earth) - cp.cross(Omega)
            VelinLDPlane    = Air_speed_body - Air_speed_body.dot(crossward_Body.T) * crossward_Body
            dragDirection = -VelinLDPlane / VelinLDPlane.norm() 
            liftDirection = -crossward_Body.cross(dragDirection)
            
        return VelinLDPlane, dragDirection, liftDirection

    def GenForceWing(A, VelinLDPlane, dragDirection, liftDirection, Cd, Cl, cp):
        # Cette fonction permet de générer les forces aerodynamique d'une aile dans son repère.
        D = A * VelinLDPlane.norm()**2 * dragDirection * Cd
        L = A * VelinLDPlane.norm()**2 * liftDirection * Cl

        F_wing = L+D 
        Torque_wing =  cp.cross(F_wing)

        return F_wing, Torque_wing

    def Generate_Sum_Force_wing(A_list, Omega, cp_list, R_list, vB, vW,  Cd_list, Cl_list, crossward_body, r_queue, r_queue_neg):
        # Cette function permet de généer l'équation complète de la somme des forces pour les différentes surfaces portantes 
        p = 0
        Sum_Force_Wing = Matrix([0,0,0])
        Sum_Torque_Wing =  Matrix([0,0,0])
        for i in cp_list:
            VelinLDPlane, dragDirection, liftDirection= GenDirectForceWing(Omega, i, vB, vW, R_list[p], crossward_body)
            if bo==True:
                F_wing, Torque_wing =  GenForceWing(A_list[p], VelinLDPlane, dragDirection, liftDirection, Cd_list[p], Cl_list[p], i)
                Sum_Force_Wing  = Sum_Force_Wing +  F_wing
                Sum_Torque_Wing = Sum_Torque_Wing + Torque_wing
                p+=1
            else:
                if p == 2 :
                    # Comme la fonction GenForceWing donne les efforts des ailes dans leur repère propre, on doit passer par les matrice de rotation pour les ailes de la queue
                    F_wing, Torque_wing =  GenForceWing(A_list[p], VelinLDPlane, dragDirection, liftDirection, Cd_list[p], Cl_list[p], Matrix([0,0,0]))
                    Sum_Force_Wing  = Sum_Force_Wing +  r_queue.T  @ F_wing 
                    Sum_Torque_Wing = Sum_Torque_Wing + i.cross(r_queue.T  @ F_wing) 
                elif p == 3 :
                    F_wing, Torque_wing =  GenForceWing(A_list[p], VelinLDPlane, dragDirection, liftDirection, Cd_list[p], Cl_list[p], Matrix([0,0,0]))
                    Sum_Force_Wing  = Sum_Force_Wing +  r_queue_neg.T  @ F_wing 
                    Sum_Torque_Wing = Sum_Torque_Wing + i.cross(r_queue_neg.T  @ F_wing) 
                else:
                    F_wing, Torque_wing =  GenForceWing(A_list[p], VelinLDPlane, dragDirection, liftDirection, Cd_list[p], Cl_list[p], i)
                    Sum_Force_Wing  = Sum_Force_Wing +  F_wing
                    Sum_Torque_Wing = Sum_Torque_Wing + Torque_wing
                p+=1
            
        return Sum_Force_Wing, Sum_Torque_Wing


    def GenForceMoteur(Omega, ct, cq, omega_rotor, cp, vB, vW, ch, R, motor_axis_in_body_frame, spinning_sense):
        ## Cette fonction calcule les effort produit par un rotor sur le drone en fonction de son sens de rotation et de sa localisation, les efforts sont donnés
        ## dans le repère inertiel. l'axe des moteur est placé suivant l'axe x du drone (mode avion seulement)
        if bo==True:
            Air_speed_earth = vB - vW
            Axial_speed = Air_speed_earth.dot(R*motor_axis_in_body_frame)
            lat_speed = Air_speed_earth - (Axial_speed * (R*motor_axis_in_body_frame))
            T = ct*omega_rotor**2

            T_vec = T * R*motor_axis_in_body_frame 

            torque_at_body_center = cp.cross(T_vec.T)
        else:
            Air_speed_earth = vB - vW
            air_speed_in_rotor_frame = (R.T* Air_speed_earth) - cp.cross(Omega)
            Axial_speed = air_speed_in_rotor_frame.dot(motor_axis_in_body_frame)
            lat_speed = air_speed_in_rotor_frame - (Axial_speed * (motor_axis_in_body_frame))
            T = ct*omega_rotor**2
            H = ch * omega_rotor

            T_vec = T * motor_axis_in_body_frame - H * lat_speed

            torque = - omega_rotor * cq * lat_speed
            torque = - spinning_sense * cq * T * motor_axis_in_body_frame 
            torque_at_body_center = torque + cp.cross(T_vec.T)
        return T_vec, torque_at_body_center

    def Generate_Sum_Force_Moteur(Omega, ct, cq, omega_rotor, cp_list_rotor, vB, vW, ch, R, motor_axis_in_body_frame_list, spinning_sense_list):
        # Calcul des forces des moteurs sur le drone, génère toutes les forces, ainsi que le couple appliqué au centre de gravité du drone, dans le repère inertiel
        p = 0
        Sum_Force_Rotor = Matrix([0,0,0])
        Sum_Torque_Rotor =  Matrix([0,0,0])
        for cp in cp_list_rotor:
            F_rotor, Q_rotor = GenForceMoteur(Omega, ct, cq, omega_rotor, cp, vB, vW, ch, R, motor_axis_in_body_frame_list, spinning_sense_list[p])        
            Sum_Force_Rotor  = Sum_Force_Rotor + F_rotor
            Sum_Torque_Rotor = Sum_Torque_Rotor + Q_rotor
            p+=1

        return Sum_Force_Rotor, Sum_Torque_Rotor

    def Compute_list_coeff(alpha_list, alpha_0_list, alpha_s, delta_0_list, delta_s, CL_1_sa, CD_1_fp, k_0, k_1, k_2, CD_0_fp, CD_0_sa, CD_1_sa):
        Cd_list = Matrix([0 for i in range(len(alpha_list))])
        Cl_list = Matrix([0 for i in range(len(alpha_list))])
        for count, alpha in enumerate(alpha_list):
            Cl_list[count],  Cd_list[count] =  compute_cl_cd(alpha, alpha_0_list[count], alpha_s, delta_0_list[count], delta_s, CL_1_sa, CD_1_fp, k_0, k_1, k_2, CD_0_fp, CD_0_sa, CD_1_sa)
        return Cl_list , Cd_list

    "liftdrag forces"
    t2=time.time()
    print("Elapsed : %f s , Prev step time: %f s \\ Solving lifrdrag model ..."%(t2-t0,t2-t1))
    # Ces équation servent uniquement pour le calcul de l'angle d'attaque pour le calcul des coeffficient aérodynamique Cd et Cl, il n'interviennent pas directement dans le calcul des efforts. 
    VelinLDPlane, dragDirection, liftDirection=  GenDirectForceWing(Omega, X_cp, v_B, v_W, R, crossward_B)                        
    Cl_list, Cd_list = Compute_list_coeff(alpha_list, alpha0_list, alpha_s, delta0_list, delta_s, CL_1_sa, CD_1_fp, k_0, k_1, k_2, CD_0_fp, CD_0_sa, CD_1_sa)
    Sum_F_wing_complete, Sum_T_wing_complete = Generate_Sum_Force_wing(Aire_list, Omega, cp_list, R_list_sympy, v_B, v_W, Cd_list, Cl_list, crossward_B, r, r_neg)

    t3=time.time()
    print("Elapsed : %f s , Prev step time: %f s \\ Solving rotor model ..."%(t3-t0,t3-t2))
    ##################### Sommes des efforts des moteurs
    Sum_F_rotor_complete, Sum_T_rotor_complete = Generate_Sum_Force_Moteur(Omega, C_t, C_q, omega_rotor, cp_list_rotor, v_B, v_W, C_h, R, motor_axis_in_body_frame, spinning_sense_list)
    Sum_F_rotor_complete.simplify()
    Sum_T_rotor_complete.simplify()
    Effort_Aero_complete = [Sum_F_wing_complete + Sum_F_rotor_complete , Sum_T_wing_complete + Sum_T_rotor_complete]

    t35=time.time()
    print("Elapsed : %f s , Prev step time: %f s \\ Solving Dynamics ..."%(t35-t0,t35-t3))

    theta=[]
    for i in (CD_0_sa,
            CD_0_fp,
            CD_1_sa,
            CL_1_sa,
            CD_1_fp,
            k_0,
            k_1,
            k_2):
        theta.append(i)

    if id_wind:
        theta.append(Vw1)
        theta.append(Vw2)
        #theta.append(Vw3)

    theta = Matrix([theta])

    ########## Equation du gradient utilisé en simulation ####################
    VelinLDPlane_function = lambdify((Omega, X_cp, v_B, v_W, R), VelinLDPlane, 'numpy')
    dragDirection_function = lambdify((Omega, X_cp, v_B, v_W, R), dragDirection, 'numpy')
    liftDirection_function = lambdify((Omega, X_cp, v_B, v_W, R), liftDirection, 'numpy')
  #  Effort_Aero_complete_function = lambdify((Aire_list, Omega, R, v_B, v_W, cp_list, alpha_list, alpha0, alpha_s, delta0_list, delta_s, \
  #                                            CL_1_sa, CD_1_fp, k_0, k_1, k_2, CD_0_fp, CD_0_sa, CD_1_sa, C_t, C_q, C_h, omega_rotor), Effort_Aero_complete, 'numpy')

    t37=time.time()
    print("Elapsed : %f s , Prev step time: %f s \\ Generating costs ..."%(t37-t0,t37-t35))

    m=symbols('m', real=True)
    g1,g2,g3=symbols('g1,g2,g3', real=True)
    g=Matrix([g1,g2,g3])
    
    #Génération des équations finales pour la gradient du csout et des RMS error
    if bo==True:
        forces = (Sum_F_wing_complete + Sum_F_rotor_complete) + m*g
    else:
        forces = R@(Sum_F_wing_complete + Sum_F_rotor_complete) + m*g

    # torque = R@(Sum_T_wing_complete + Sum_T_rotor_complete)

    new_acc = forces/m
    new_v = v_B + new_acc*dt

    alog_i,alog_j,alog_k=symbols("alog_i,alog_j,alog_k",real=True)
    alog=Matrix([[alog_i],[alog_j],[alog_k]])

    err_a=Matrix(alog-new_acc)
    err_v=Matrix(v_log-new_v)

#     cost_scaler_a=symbols('C_sa',real=True,positive=True)
#     cost_scaler_v=symbols('C_sv',real=True,positive=True)

#     sqerr_a=Matrix([1.0/cost_scaler_a*(err_a[0,0]**2+err_a[1,0]**2+err_a[2,0]**2)])
#     sqerr_v=Matrix([1.0/cost_scaler_v*(err_v[0,0]**2+err_v[1,0]**2+err_v[2,0]**2)])
    
    
    sqerr_a=Matrix([err_a[0,0]**2+err_a[1,0]**2+err_a[2,0]**2])
    sqerr_v=Matrix([err_v[0,0]**2+err_v[1,0]**2+err_v[2,0]**2])

    t38=time.time()
    print("Elapsed : %f s , Prev step time: %f s \\ Generating Jacobian ..."%(t38-t0,t38-t37))

    Ja=sqerr_a.jacobian(theta)
    Jv=sqerr_v.jacobian(theta)

    Y=Matrix([new_acc,new_v,sqerr_a,sqerr_v,Ja.T,Jv.T])

    X =(alog,v_log,dt, Aire_list, Omega, R, v_pred, v_W, cp_list, alpha_list, alpha0, alpha_s, delta0_list, delta_s, \
                                CL_1_sa, CD_1_fp, k_0, k_1, k_2, CD_0_fp, CD_0_sa, CD_1_sa, C_t, C_q, C_h, omega_rotor, \
                                  g, m)

    function_physical_model =[VelinLDPlane_function, \
                             dragDirection_function, \
                             liftDirection_function,\
                             compute_alpha]
    model_func=lambdify(X,Y, modules='numpy')

    import dill as dill
    dill.settings['recurse'] = True

    dill.dump([model_func, function_physical_model], open("./.Funcs/model_func_"+str(used_logged_v_in_model), "wb"))
    print("Elapsed : %f s , Prev step time: %f s \\ Finished ..."%(time.time()-t0,time.time()-t3))



Elapsed : 0.001812 s , Prev step time: -1 s \ Generating first symbols ...
Elapsed : 0.056512 s , Prev step time: -1 s \ Generating dynamics function ...
Elapsed : 0.056570 s , Prev step time: 0.000058 s \ Solving lifrdrag model ...
Elapsed : 0.263004 s , Prev step time: 0.206434 s \ Solving rotor model ...
Elapsed : 0.529950 s , Prev step time: 0.266947 s \ Solving Dynamics ...
Elapsed : 0.695315 s , Prev step time: 0.165365 s \ Generating costs ...
Elapsed : 0.721391 s , Prev step time: 0.026076 s \ Generating Jacobian ...
Elapsed : 9.107569 s , Prev step time: 8.844566 s \ Finished ...

Elapsed : 0.000286 s , Prev step time: -1 s \ Generating first symbols ...
Elapsed : 0.016120 s , Prev step time: -1 s \ Generating dynamics function ...
Elapsed : 0.016261 s , Prev step time: 0.000141 s \ Solving lifrdrag model ...
Elapsed : 0.209592 s , Prev step time: 0.193331 s \ Solving rotor model ...
Elapsed : 0.271070 s , Prev step time: 0.061478 s \ Solving Dynamics ...
Elapsed : 0.480211 s